In [1]:
import torch

torch.__version__

'2.7.1+cpu'

# Version 0.1

## 从Colab中下载best.pt文件

In [ ]:
from google.colab import files
files.download('/content/best.pt')

## 安装依赖

In [ ]:
pip install ultralytics flask #ultralytics支持YOLOv8

## Flask后端代码

保存为 `app.py`

In [ ]:
from flask import Flask, request, render_template, send_file
import os
from ultralytics import YOLO
import uuid

app = Flask(__name__)

# 加载模型
model = YOLO("best.pt")  # 把你的模型路径写上

# 上传图像目录
UPLOAD_FOLDER = 'uploads'
RESULT_FOLDER = 'results'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(RESULT_FOLDER, exist_ok=True)

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']
        if file:
            filename = str(uuid.uuid4()) + ".jpg"
            filepath = os.path.join(UPLOAD_FOLDER, filename)
            file.save(filepath)

            # 推理
            results = model.predict(filepath, save=True, project=RESULT_FOLDER, name='pred', exist_ok=True)
            result_path = results[0].save_dir + "/" + filename

            return send_file(result_path, mimetype='image/jpeg')

    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Tengyue Wu\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

d:\anaconda3\envs\cv311\Lib\site-packages\IPython\core\interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


```log
Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Tengyue Wu\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
 * Serving Flask app '__main__'
 * Debug mode: on
WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
An exception has occurred, use %tb to see the full traceback.

SystemExit: 1
d:\anaconda3\envs\cv311\Lib\site-packages\IPython\core\interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
```

出现这个提示和错误，说明 `Flask` 本身没问题，成功跑起来了；但是由于是在 `Jupyter Notebook / IPython` 里执行了 `app.run()` ，所以 `Python` 的主线程和 `IPython` 的事件循环冲突了，导致 `SystemExit` 。

出现这个情况的原因是，当在 `IPython`（例如 `Colab`, `Jupyter Notebook`, `Spyder` 或某些 `IDE` 的交互 `Console`） 中运行 `Flask` 的 `app.run()` 时，`Flask` 会尝试调用 `sys.exit()` 来结束调试模式或者重启进程，这就和交互式环境冲突了。

解决办法是，保存为 `app.py` 文件，然后在 `bash` 里运行，这样就完全避开了 `Notebook` 的环境干扰，`Flask` 会正常启动。

## HTML页面

In [ ]:
<!doctype html>
<html lang="en">
  <head>
    <title>图像识别上传</title>
  </head>
  <body>
    <h1>上传图片进行识别</h1>
    <form method="POST" enctype="multipart/form-data">
      <input type="file" name="file" accept="image/*">
      <input type="submit" value="上传并识别">
    </form>
  </body>
</html>

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>YOLOv8 Demo</title>
</head>
<body>
    <h1>Upload an Image</h1>
    <form action="/" method="post" enctype="multipart/form-data">
        <input type="file" name="file">
        <input type="submit" value="Upload">
    </form>
</body>
</html>

## 运行应用

In [ ]:
python app.py

浏览器访问： `http://127.0.0.1:5000` ，上传一张图片，就能得到模型推理后的结果图像。

当运行
```bash
python app.py
```
或者
```python
app.run(debug=True)
```
Flask 会：

* 开启一个开发服务器

* 默认监听 http://127.0.0.1:5000

关闭时：

1. 只关闭浏览器页

   * 只是关闭浏览器标签页，并不会自动关闭 Flask 进程。

   * Flask 服务还在后台运行，端口依然被占用。

2. 正确的关闭方式

   * 需要回到运行 Flask 的终端，按 Ctrl + C，这样 Flask 会正常关闭，端口才会彻底释放。

若没关闭干净，再次运行Flask会报错：
```makefile
OSError: [Errno 98] Address already in use
```
意思是端口还在被占用。

端口被占用时：

找出哪个进程占用了 5000 端口，然后在安全的情况下杀掉：

* Windows：
```bash
netstat -ano | findstr :5000
```
然后查看PID，杀掉进程：
```bash
taskkill /PID 进程号 /F
```
* Linux/macOS:
```bash
lsof -i :5000
kill -9 进程号
```

## 打包整个项目成为 `.exe` 文件

### 准备文件结构

In [ ]:
yolo_app/
├── app.py
├── best.pt
├── templates/
│   └── index.html

In [ ]:
your_project/
│
├── app.py          # 你的 Flask 主程序
├── best.pt         # 你的 YOLO 模型文件
├── templates/
│   └── index.html  # 你的上传页面
├── uploads/        # 上传图片保存文件夹
├── results/        # 检测结果保存文件夹
└── other files ...

### 安装pyinstaller

In [ ]:
pip install pyinstaller

### 使用PyInstaller打包成为EXE

In [ ]:
pyinstaller --onefile --hidden-import torch --add-data "templates;templates" --add-data "best.pt;." app.py

In [ ]:
pyinstaller -F --add-data "templates;templates" --add-data "best.pt;." app.py

上面这个命令的意思是：

* `pyinstaller`：调用 `PyInstaller` 这个打包工具。

* `-F`：表示打包成单个独立的可执行文件（single file），生成一个 `.exe`。

* `app.py`：你要打包的 `Python` 脚本。

* `--add-data "best.pt;."` ：

    * `best.pt` 是你项目里的模型文件。

    * `.;` 这里是“目的路径”，表示把 `best.pt` 放到生成的 `exe` 同目录下。

    * 也就是说，打包时把 `best.pt` 文件包含进来，运行时会被解压到临时目录或者 `exe` 旁边，代码用 `resource_path()` 找到它。

* `--add-data "templates;templates"` ：

    * `templates` 是你 `Flask` 项目的模板文件夹。

    * 目的路径同名 `templates` 文件夹。

    * 这样，`templates` 文件夹也会被打包进去，运行时依然能用 `templates` 路径访问到里面的 `HTML` 文件。

#### PyInstaller执行命令涉及到的文件

**主要输入**

* Python 脚本（ `.py` ）

    例如： `app.py`

    这是你打包的入口文件。

* 依赖库

    `PyInstaller` 会自动解析 `import` 的库，并把它们收集到打包结果中。

    如果你有额外的非 `Python` 文件（如 `.json`、`.yaml`、图片、模型文件等），需要在命令里用 `--add-data` 明确告诉 `PyInstaller`。

#### PyInstaller执行命令后生成的文件和文件夹

**`app.spec`**

作为配置文件，记录了打包细节（入口、数据、图标等）。可以修改后重复使用。

**`build/`**

临时构建的文件夹，放置中间产物（分析依赖、解包库、缓存编译等）。

**`dist/`**

打包结果文件夹。最终可执行文件的位置。如果是 `--onefile`，只有一个 `exe`；如果是默认多文件格式，会有一个同名文件夹和一堆依赖。

#### 结果举例

* 默认模式（多文件）
    
    执行
    ```bash
    pyinstaller app.py
    ```
    生成
    ```bash
    .
    ├── app.spec         # 配置文件
    ├── build/            # 中间文件
    │   └── ...
    ├── dist/
    │   └── app/         # 输出文件夹
    │       ├── app.exe  # 可执行文件
    │       ├── ...       # 依赖库和DLL
    └── app.py           # 源代码
    ```
    `PyInstaller` 默认是多文件模式（`--onedir`），意思是：

  * 可执行文件和所有依赖（`Python` 解释器、库、动态链接库 `.dll` 或 `.so`、资源文件等）都分开放到一个文件夹里。

  * 这个文件夹名字通常就是你入口脚本的名字。

  * 例子：
    ```css
    .
    ├── main.py
    ├── main.spec
    ├── build/
    │   └── ...
    └── dist/
        └── main/          ← 这个文件夹名字跟入口文件名一样
            ├── main.exe   ← 可执行文件
            ├── python39.dll
            ├── library.zip
            ├── vcruntime140.dll
            ├── 其他依赖的 DLL、PYC、库文件...
    ```
  * 特点：
    | 项目                   |说明                                                         |
    | :-------------------: | :---------------------------------------------------------: |
    | `main.exe`            | 这是真正的启动程序                                            |
    | 其他 `DLL` / `zip` / 文件  | `Python` 运行时、标准库、第三方库，`PyInstaller` 把它们单独拷贝出来 |
    | 启动时                 | `main.exe` 会动态加载同目录下的库                             |
    | 可移植性               | 整个 `main/` 文件夹必须 **完整一起拷贝** 才能运行，缺一不可    |
  
  * 为什么需要多文件？
    * 启动速度快
  
        （相比 --onefile，不用先解压再运行）

    * 排查问题方便
        
        出错时可以直接看哪些依赖出了问题

    * 避免某些杀毒软件对单文件打包的误报

* 单文件模式
    
    执行
    ```bash
    pyinstaller --onefile app.py
    ```
    或者
    ```bash
    pyinstaller -F app.py
    ```
    两者命令等价，`-F` 就是 `--onefile` 的缩写，都是“单文件模式”。
    
    生成
    ```bash
    .
    ├── app.spec
    ├── build/
    │   └── ...
    ├── dist/
    │   └── app.exe   # 仅一个文件，所有依赖打包进去了
    └── app.py
    ```
  * 为什么需要单文件？

    * 需要只分发一个 `.exe`

    * 不在乎第一次启动时要解压临时文件

#### spec文件

`spec` 文件是 `PyInstaller` 的核心高级用法之一，很多场景下比命令行更灵活。其本质就是一个 `Python` 脚本，它是 `PyInstaller` 的配置文件，它包括：
* 打包哪个入口

* 加载哪些额外文件

* 输出名字是什么

* 是单文件还是多文件

* 图标、版本信息等等

**一个典型示例**

假设我的文件结构如下：
```css
main.py
config.yaml
icon.ico
```
我需要：
* 打包成单文件

* 加上图标

* 打包时把 `config.yaml` 一起打进去

则下面是对应的 `main.spec` :
```python
# -*- mode: python ; coding: utf-8 -*-

block_cipher = None

a = Analysis(
    ['main.py'],          # 入口脚本
    pathex=[],            # 可以写额外的搜索路径
    binaries=[],          # 要打包的二进制文件
    datas=[('config.yaml', '.')],
    hiddenimports=[],     # 隐式导入的库
    hookspath=[],         # 自定义 hook 路径
    runtime_hooks=[],
    excludes=[],
    win_no_prefer_redirects=False,
    win_private_assemblies=False,
    cipher=block_cipher,
)

pyz = PYZ(a.pure, a.zipped_data, cipher=block_cipher)

exe = EXE(
    pyz,
    a.scripts,
    [],
    exclude_binaries=True,
    name='main',          # 输出 exe 的名字
    debug=False,
    bootloader_ignore_signals=False,
    strip=False,
    upx=True,
    console=True,         # True: 有黑框；False: 没黑框
    icon='icon.ico'       # 图标
)

coll = COLLECT(
    exe,
    a.binaries,
    a.zipfiles,
    a.datas,
    strip=False,
    upx=True,
    upx_exclude=[],
    name='main'
)
```
**使用方法**

1. 生成一个模板
    
    ```bash
    pyinstaller main.py
    ```
    它会生成 `main.spec` 文件（用于记录这次的配置）。同时使用默认参数打包 `main.py` 文件（默认：多文件模式、有控制台、无图标）。

2. 打开 `.spec` 修改内容
   
    根据你的需求：
    * 改 datas

    * 改 name

    * 改 icon

    * 改 console 选项等

    以后如果再次执行 `pyinstaller main.py`，`PyInstaller` 不会用你修改过的 `main.spec`，而是重新根据命令行参数走一遍。

3. 用 `.spec` 重新打包

    ```bash
    pyinstaller main.spec
    ```
    `PyInstaller` 就会完全按照 `.spec` 里的设置来执行，而不是用命令行参数（会被忽略）。

**小技巧**

* ✅ datas=[('文件', '目标文件夹')]

    格式是：
    ```python
    datas = [
        ('config.yaml', '.'), 
        ('data/*.json', 'data')
    ]
    ```
* ✅ `console=False`

    如果你是 `GUI` 程序，不想要黑框，就把 `console=True` 改成 `False`。

* ✅ 多个文件入口
    
    如果是多入口（多 `exe`），可以写多个 `EXE`，最后放到 `COLLECT` 里。


### 生成的文件

In [ ]:
dist/
└── app.exe

### 运行打包后的EXE

In [ ]:
./dist/app.exe

在浏览器中访问 `http://127.0.0.1:5000` 就能上传图片看到检测结果了

# Version 0.2

## 优化HTML页面

使用简单、好看、功能完整的 `index.html` ，用于上传图片并显示预测结果。

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>YOLOv8 检测页面</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background: #f2f2f2;
            text-align: center;
            padding: 50px;
        }
        h1 {
            color: #333;
        }
        form {
            margin: 20px auto;
            padding: 20px;
            background: #fff;
            border-radius: 8px;
            width: 400px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }
        input[type="file"] {
            margin: 20px 0;
        }
        input[type="submit"] {
            background: #4CAF50;
            color: white;
            border: none;
            padding: 10px 20px;
            border-radius: 4px;
            cursor: pointer;
        }
        input[type="submit"]:hover {
            background: #45a049;
        }
        img {
            margin-top: 30px;
            max-width: 80%;
            border-radius: 8px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }
    </style>
</head>
<body>
    <h1>YOLOv8 图片检测</h1>
    <form method="POST" enctype="multipart/form-data">
        <label for="file">选择一张图片：</label><br>
        <input type="file" name="file" id="file" accept="image/*" required><br>
        <input type="submit" value="上传并检测">
    </form>

    {% if request.method == 'POST' %}
        <h2>检测结果：</h2>
        <img src="{{ url_for('static', filename='result.jpg') }}" alt="检测结果">
    {% endif %}
</body>
</html>

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>YOLOv8 图片检测</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background: #f2f2f2;
            text-align: center;
            padding: 50px;
        }
        h1 {
            color: #333;
        }
        form {
            display: inline-block;
            margin-top: 20px;
            padding: 30px 40px;
            background: #fff;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            text-align: center;
        }
        input[type="file"] {
            display: block;
            margin: 20px auto;
        }
        input[type="submit"] {
            background: #4CAF50;
            color: white;
            border: none;
            padding: 12px 30px;
            border-radius: 4px;
            cursor: pointer;
            font-size: 16px;
        }
        input[type="submit"]:hover {
            background: #45a049;
        }
        img {
            margin-top: 40px;
            max-width: 80%;
            border-radius: 8px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }
    </style>
</head>
<body>
    <h1>YOLOv8 图片检测</h1>
    <form method="POST" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*" required>
        <input type="submit" value="上传并检测">
    </form>

    {% if request.method == 'POST' %}
        <h2>检测结果：</h2>
        <img src="{{ url_for('static', filename='result.jpg') }}" alt="检测结果">
    {% endif %}
</body>
</html>

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>YOLOv8 图片检测</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      background: #f2f2f2;
      height: 100vh;
      margin: 0;
      display: flex;
      justify-content: center; /* 水平居中 */
      align-items: center;     /* 垂直居中 */
      flex-direction: column;
    }
    h1 {
      color: #333;
      margin-bottom: 30px;
    }
    form {
      background: white;
      padding: 30px 40px;
      border-radius: 10px;
      box-shadow: 0 2px 10px rgba(0,0,0,0.1);
      display: flex;
      flex-direction: column;
      align-items: center; /* 子元素垂直居中 */
      width: 320px;        /* 统一宽度 */
    }
    input[type="file"] {
      width: 100%;         /* 占满form宽度 */
      margin-bottom: 20px;
      cursor: pointer;
    }
    input[type="submit"] {
      width: 100%;
      background: #4CAF50;
      color: white;
      border: none;
      padding: 12px 0;
      border-radius: 4px;
      font-size: 16px;
      cursor: pointer;
      transition: background-color 0.3s ease;
    }
    input[type="submit"]:hover {
      background: #45a049;
    }
  </style>
</head>
<body>
  <h1>YOLOv8 图片检测</h1>
  <form method="POST" enctype="multipart/form-data">
    <input type="file" name="file" accept="image/*" required />
    <input type="submit" value="上传并检测" />
  </form>
</body>
</html>

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8" />
<title>YOLOv8 图片检测</title>
<style>
  body {
    font-family: Arial, sans-serif;
    background: #f2f2f2;
    height: 100vh;
    margin: 0;
    display: flex;
    justify-content: center; /* 水平居中 */
    align-items: center;     /* 垂直居中 */
    flex-direction: column;
  }
  h1 {
    color: #333;
    margin-bottom: 30px;
  }
  form {
    background: white;
    padding: 30px 40px;
    border-radius: 10px;
    box-shadow: 0 2px 10px rgba(0,0,0,0.1);
    display: flex;
    flex-direction: column;
    align-items: center;
    width: 320px;
  }

  /* 隐藏真实文件输入框 */
  input[type="file"] {
    display: none;
  }

  /* 自定义文件选择按钮 */
  .file-label {
    width: 100%;
    padding: 12px 0;
    background: #2196F3;
    color: white;
    text-align: center;
    border-radius: 4px;
    font-size: 16px;
    cursor: pointer;
    margin-bottom: 20px;
    user-select: none;
    transition: background-color 0.3s ease;
  }
  .file-label:hover {
    background: #1976D2;
  }

  input[type="submit"] {
    width: 100%;
    background: #4CAF50;
    color: white;
    border: none;
    padding: 12px 0;
    border-radius: 4px;
    font-size: 16px;
    cursor: pointer;
    transition: background-color 0.3s ease;
  }
  input[type="submit"]:hover {
    background: #45a049;
  }
</style>
</head>
<body>
  <h1>YOLOv8 图片检测</h1>
  <form method="POST" enctype="multipart/form-data">
    <!-- label绑定file输入 -->
    <label for="file-upload" class="file-label">选择文件</label>
    <input id="file-upload" type="file" name="file" accept="image/*" required />

    <input type="submit" value="上传并检测" />
  </form>
</body>
</html>

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8" />
<title>YOLOv8 图片检测</title>
<style>
  body {
    font-family: Arial, sans-serif;
    background: #f2f2f2;
    height: 100vh;
    margin: 0;
    display: flex;
    justify-content: center; 
    align-items: center;    
    flex-direction: column;
  }
  h1 {
    color: #333;
    margin-bottom: 30px;
  }
  form {
    background: white;
    padding: 30px 40px;
    border-radius: 10px;
    box-shadow: 0 2px 10px rgba(0,0,0,0.1);
    display: flex;
    flex-direction: column;
    align-items: center;
    width: 320px;
  }

  input[type="file"] {
    display: none;
  }

  .file-label {
    width: 100%;
    padding: 12px 0;
    background: #2196F3;
    color: white;
    text-align: center;
    border-radius: 4px;
    font-size: 16px;
    cursor: pointer;
    user-select: none;
    transition: background-color 0.3s ease;
  }
  .file-label:hover {
    background: #1976D2;
  }

  #file-name {
    margin-top: 8px;
    font-size: 14px;
    color: #555;
    min-height: 20px;
    text-align: center;
    word-break: break-all;
  }

  input[type="submit"] {
    width: 100%;
    background: #4CAF50;
    color: white;
    border: none;
    padding: 12px 0;
    border-radius: 4px;
    font-size: 16px;
    cursor: pointer;
    margin-top: 20px;
    transition: background-color 0.3s ease;
  }
  input[type="submit"]:hover {
    background: #45a049;
  }
</style>
</head>
<body>
  <h1>YOLOv8 图片检测</h1>
  <form method="POST" enctype="multipart/form-data">
    <label for="file-upload" class="file-label">选择文件</label>
    <input id="file-upload" type="file" name="file" accept="image/*" required />
    <div id="file-name"></div>
    <input type="submit" value="上传并检测" />
  </form>

  <script>
    const fileInput = document.getElementById('file-upload');
    const fileNameDiv = document.getElementById('file-name');

    fileInput.addEventListener('change', () => {
      if (fileInput.files.length > 0) {
        fileNameDiv.textContent = fileInput.files[0].name;
      } else {
        fileNameDiv.textContent = '';
      }
    });
  </script>
</body>
</html>

# Version 0.3

## 使用webbrowser模块

可以在 Flask 运行后，自动用 Python 调用浏览器打开对应的 URL

将原来的 `if __name__ == '__main__':` 里改成如下代码：

In [ ]:
import webbrowser
from threading import Timer

if __name__ == '__main__':
    port = 5000
    url = f"http://127.0.0.1:{port}/"

    # 启动 Flask 服务器的同时，延迟几秒打开浏览器
    def open_browser():
        webbrowser.open_new(url)

    Timer(1, open_browser).start()
    app.run(debug=True, port=port)


这里：

* 用 `threading.Timer` 延迟 1 秒后打开浏览器，确保服务器先启动

* 自动打开默认浏览器并访问 Flask 服务地址

运行该脚本，程序启动后浏览器会自动弹出并打开你的网页，不用自己复制粘贴URL了。

如果用的是别的端口，记得改 `port` 变量对应的端口号。

# Version 0.4

一次上传多张图片（该分支未融合）

## 修改 `input` 为支持多选

In [ ]:
<input type="file" name="file" id="file" accept="image/*" multiple required>

## 修改Flask后端

后端要从 `request.files.getlist()` 获取多张图片，然后对每张图片循环保存、推理，最后把所有推理后的结果打包（或只返回第一张结果）。

In [ ]:
from flask import Flask, request, render_template, send_file, send_from_directory, redirect, url_for
import os
from ultralytics import YOLO
import uuid
import zipfile

app = Flask(__name__)

# 加载模型
model = YOLO("best.pt")

# 上传和结果文件夹
UPLOAD_FOLDER = 'uploads'
RESULT_FOLDER = 'results'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(RESULT_FOLDER, exist_ok=True)

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        files = request.files.getlist('file')
        saved_files = []

        for file in files:
            if file:
                filename = str(uuid.uuid4()) + ".jpg"
                filepath = os.path.join(UPLOAD_FOLDER, filename)
                file.save(filepath)

                # 推理
                results = model.predict(
                    filepath,
                    save=True,
                    project=RESULT_FOLDER,
                    name='pred',
                    exist_ok=True,
                    conf=0.2,
                    show_conf=False,
                    line_width=3,
                    iou=0.2
                )
                result_path = os.path.join(results[0].save_dir, filename)
                saved_files.append(result_path)

        # 如果只上传了一张图片，直接返回该图片
        if len(saved_files) == 1:
            return send_file(saved_files[0], mimetype='image/jpeg')

        # 如果上传了多张图片，打包成 ZIP
        zip_filename = f"{uuid.uuid4()}.zip"
        zip_path = os.path.join(RESULT_FOLDER, zip_filename)
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for file_path in saved_files:
                arcname = os.path.basename(file_path)
                zipf.write(file_path, arcname)

        return send_file(zip_path, mimetype='application/zip', as_attachment=True)

    return render_template('index.html')

if __name__ == '__main__':
    import webbrowser
    from threading import Timer

    port = 5000
    url = f"http://127.0.0.1:{port}/"

    def open_browser():
        webbrowser.open_new(url)

    Timer(1, open_browser).start()
    app.run(debug=True, port=port)

## 前端按钮无需更改

前面已经在 HTML 里把 `<input>` 改成 `multiple`，按钮和表单自动就能支持批量上传。

## 工作流程

* 前端选多张图片，点击上传

* 后端循环保存每张，逐张推理

* 如果只有 1 张，直接返回单张结果

* 如果有多张，就打包 ZIP 返回

这样就可以用一个页面批量上传，一次性拿回所有结果。

# Version 0.5

## 读取解压后的路径

按照前面的打包命令，运行生成的 `.exe` 文件会报错如下：
```log
Traceback (most recent call last):
  File "app.py", line 11, in <module>
  File "ultralytics\models\yolo\model.py", line 79, in __init__
  File "ultralytics\engine\model.py", line 151, in __init__
  File "ultralytics\engine\model.py", line 295, in _load
  File "ultralytics\nn\tasks.py", line 1548, in attempt_load_one_weight
  File "ultralytics\nn\tasks.py", line 1446, in torch_safe_load
  File "ultralytics\utils\patches.py", line 117, in torch_load
  File "torch\serialization.py", line 1479, in load
  File "torch\serialization.py", line 759, in _open_file_like
  File "torch\serialization.py", line 740, in __init__
FileNotFoundError: [Errno 2] No such file or directory: 'best.pt'
[PYI-16312:ERROR] Failed to execute script 'app' due to unhandled exception!
```
这个报错是说打包后的程序找不到 `best.pt` 型文件，导致程序启动时加载模型失败。

**原因**

* 你的代码里：model = YOLO("best.pt")

* 运行时它默认从当前工作目录找 best.pt

* 打包成 exe 后，best.pt 不会自动包含进去，也不在 exe 所在目录，导致找不到文件

**解决方案**

### 在 `PyInstaller` 打包时，添加模型文件（`best.pt`）到 `exe` 里

用 `--add-data` 参数，把模型一起打包进去。

即用之前的打包命令：

In [ ]:
pyinstaller -F --add-data "templates;templates" --add-data "best.pt;." app.py

* 注意分号 `;` 是 `Windows` 下分隔符，`Linux/macOS` 下是冒号 `:`

* 这样 `templates` 和 `best.pt` 会被放到 `exe` 同目录里

### 代码中正确读取打包后的资源路径

打包后，直接用 `best.pt` 可能找不到，因为打包成 `exe` 时文件被解包到临时目录（如 `sys._MEIPASS` ）。
    
你需要用下面代码，兼容打包后读取路径：

In [ ]:
import sys
import os

def resource_path(relative_path):
    """获取打包后或未打包时资源的绝对路径"""
    if hasattr(sys, '_MEIPASS'):
        base_path = sys._MEIPASS
    else:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, relative_path)

model_path = resource_path("best.pt")
model = YOLO(model_path)

### 重新打包

* 确保 `--add-data "best.pt;."` 已添加

* 代码中使用上面 `resource_path` 读取模型路径

* 重新打包生成 `exe`

* 运行 `exe` 时，就能正确加载模型了

# Version 1.0

最终测试版本，移植后还是有路径读取问题。

In [ ]:
from flask import Flask, request, render_template, send_file
import sys
import os
from ultralytics import YOLO
import uuid
import webbrowser
from threading import Timer

app = Flask(__name__)

def resource_path(relative_path):
    """获取打包后或未打包时资源的绝对路径"""
    if hasattr(sys, '_MEIPASS'):
        base_path = sys._MEIPASS
    else:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, relative_path)

model_path = resource_path("best.pt")
model = YOLO(model_path)
# 加载模型
# model = YOLO("best.pt")

# 上传和结果文件夹
UPLOAD_FOLDER = 'uploads'
RESULT_FOLDER = 'results'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(RESULT_FOLDER, exist_ok=True)


@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']
        if file:
            filename = f"{uuid.uuid4()}.jpg"
            filepath = os.path.join(UPLOAD_FOLDER, filename)
            file.save(filepath)

            # 推理
            results = model.predict(
                filepath,
                iou=0.2,
                conf=0.5,
                show_labels=False,
                show_conf=False,
                line_width=3,
                nms=True,
                save=True,
                project=RESULT_FOLDER,
                name='pred',
                exist_ok=True
            )
            # 结果文件路径
            result_path = os.path.join(results[0].save_dir, filename)

            # 用 resource_path 转换绝对路径
            result_path = resource_path(result_path)

            print("Sending file:", result_path)
            print("Exists:", os.path.exists(result_path))

            return send_file(result_path, mimetype='image/jpeg')

    return render_template('index.html')


if __name__ == '__main__':
    port = 5000
    url = f"http://127.0.0.1:{port}/"

    def open_browser():
        webbrowser.open_new(url)

    # 只在主进程打开浏览器
    if os.environ.get("WERKZEUG_RUN_MAIN") != "true":
        Timer(1, open_browser).start()

    app.run(debug=True, port=port)


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8" />
<title>YOLOv8 图片检测</title>
<style>
  body {
    font-family: Arial, sans-serif;
    background: #f2f2f2;
    height: 100vh;
    margin: 0;
    display: flex;
    justify-content: center; 
    align-items: center;    
    flex-direction: column;
  }
  h1 {
    color: #333;
    margin-bottom: 30px;
  }
  form {
    background: white;
    padding: 30px 40px;
    border-radius: 10px;
    box-shadow: 0 2px 10px rgba(0,0,0,0.1);
    display: flex;
    flex-direction: column;
    align-items: center;
    width: 320px;
  }

  input[type="file"] {
    display: none;
  }

  .file-label {
    width: 100%;
    padding: 12px 0;
    background: #2196F3;
    color: white;
    text-align: center;
    border-radius: 4px;
    font-size: 16px;
    cursor: pointer;
    user-select: none;
    transition: background-color 0.3s ease;
  }
  .file-label:hover {
    background: #1976D2;
  }

  #file-name {
    margin-top: 8px;
    font-size: 14px;
    color: #555;
    min-height: 20px;
    text-align: center;
    word-break: break-all;
  }

  input[type="submit"] {
    width: 100%;
    background: #4CAF50;
    color: white;
    border: none;
    padding: 12px 0;
    border-radius: 4px;
    font-size: 16px;
    cursor: pointer;
    margin-top: 20px;
    transition: background-color 0.3s ease;
  }
  input[type="submit"]:hover {
    background: #45a049;
  }
</style>
</head>
<body>
  <h1>YOLOv8 图片检测</h1>
  <form method="POST" enctype="multipart/form-data">
    <label for="file-upload" class="file-label">选择文件</label>
    <input id="file-upload" type="file" name="file" accept="image/*" required />
    <div id="file-name"></div>
    <input type="submit" value="上传并检测" />
  </form>

  <script>
    const fileInput = document.getElementById('file-upload');
    const fileNameDiv = document.getElementById('file-name');

    fileInput.addEventListener('change', () => {
      if (fileInput.files.length > 0) {
        fileNameDiv.textContent = fileInput.files[0].name;
      } else {
        fileNameDiv.textContent = '';
      }
    });
  </script>
</body>
</html>
